<center><img src="https://javier.rodriguez.org.mx/itesm/2014/tecnologico-de-monterrey-blue.png" width="450" align="center"></center>  
<br><p><center><h1><b>Object Detection with Neural Network Approach: Analyzing the CIFAR-10 Dataset</b></h1></center></p>  
<p><center><h3>Course: <i>Neural Network Design and Deep Learning</i></h3></center></p>  
<p><center><h4>Instructed by: <i>Dr. Leonardo Mauricio Cañete Sifuentes</i></h4></center></p>  

<p style="text-align: right;">Alejandro Santiago Baca Eyssautier - A01656580</p>  
<p style="text-align: right;">André Colín Avila - A01657474</p>  
<p style="text-align: right;">Santiago Caballero - A01657699</p>  
<p style="text-align: right;"><i>November 28th, 2024</i></p><br>  

<br><p><h3><b>1. Introduction</b></h3></p>  

The objective of this project is to explore object detection using neural networks by analyzing the **CIFAR-10 Dataset**. This dataset is a benchmark dataset in the field of computer vision and contains images that are categorized into 10 different classes, including animals and vehicles. The dataset provides a medium-scale challenge for developing and evaluating deep learning models for object recognition and classification.  

Throughout this project, the team aims to preprocess the data, construct multiple neural network architectures, and evaluate their performance to identify the most efficient and accurate model. The project is divided into individual and team contributions, ensuring a collaborative yet personalized approach to model development.  

By leveraging deep learning techniques, the team seeks to tackle the challenges of distinguishing diverse object classes under varying conditions while maintaining computational efficiency.  

<br>  

<br><p><h3><b>2. Dataset Selection and Justification</b></h3></p>  

The **CIFAR-10 Dataset** consists of 60,000 images in 10 classes, with 6,000 images per class. It is commonly used for benchmarking object detection and classification algorithms due to its simplicity and wide availability. Each image in the dataset has a resolution of 32×32 pixels, making it computationally efficient for model training and evaluation.  

**Key Features:**  

- **Name**: CIFAR-10 Dataset  
- **Download URL**: [CIFAR-10 Dataset on Papers with Code](https://paperswithcode.com/dataset/cifar-10)  
- **Description**: The dataset includes 10 classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck. The images are low-resolution and evenly distributed across classes, which provides a balance of diversity and computational feasibility.  

**Justification**  

The CIFAR-10 Dataset is an ideal choice for this project for several reasons:  

1. **Problem Relevance**: The dataset aligns with the team's objective of solving an object detection problem. Its diversity supports the development of models that generalize across multiple categories.  
2. **Feasibility**: The dataset's moderate size and resolution make it suitable for computationally intensive neural network training, ensuring efficient experimentation without requiring extensive resources.  
3. **Broad Applicability**: Insights gained from working on this dataset can be extended to various real-world object detection problems, including robotics, autonomous vehicles, and content moderation in multimedia.  

<br>  

<br><p><h3> <b>3. Data Preprocessing and Splitting</b></h3></p>

The CIFAR-10 dataset is loaded using TensorFlow's built-in `tf.keras.datasets.cifar10` module, which provides direct access to the dataset. This dataset is pre-divided into training and test sets, simplifying the data preparation process. The project further splits the training set into training and validation subsets for hyperparameter tuning and evaluation during model development.

**Preprocessing Steps**

1. **Dataset Loading**: The CIFAR-10 dataset is loaded directly into memory, providing `train_images`, `train_labels`, `test_images`, and `test_labels`.
2. **Dataset Splitting**:
   - The training set is split into **training** (80%) and **validation** (20%) subsets.
3. **Normalization**: Pixel values are normalized to the range [0, 1] to stabilize and accelerate training.
4. **Data Augmentation**: Techniques such as random flipping and rotations are applied to improve model generalization.

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Split the training data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)

# Normalize pixel values to the range [0, 1]
train_images = train_images.astype("float32") / 255.0
val_images = val_images.astype("float32") / 255.0
test_images = test_images.astype("float32") / 255.0

# Output dataset sizes
print(f"Training set size: {len(train_images)} samples")
print(f"Validation set size: {len(val_images)} samples")
print(f"Test set size: {len(test_images)} samples")

2024-11-29 04:29:03.933351: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Training set size: 40000 samples
Validation set size: 10000 samples
Test set size: 10000 samples


<br>

**Output Explanation**:
- Training set: 40,000 samples (80% of the original training set)
- Validation set: 10,000 samples (20% of the original training set)
- Test set: 10,000 samples (predefined by CIFAR-10)

**Data Augmentation**  
To improve the model's ability to generalize to unseen data, data augmentation techniques are applied to the training set:

1. **Random Flipping**: Horizontally flips images with a 50% probability.
2. **Random Rotation**: Rotates images by a small angle to simulate diverse viewing perspectives.

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the augmentation pipeline
data_augmentation = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=10  # Rotate images by up to 10 degrees
)

# Fit the augmentation pipeline to the training data
train_generator = data_augmentation.flow(train_images, train_labels, batch_size=32)
val_generator = ImageDataGenerator().flow(val_images, val_labels, batch_size=32)
test_generator = ImageDataGenerator().flow(test_images, test_labels, batch_size=32)

<br>

By leveraging TensorFlow's `ImageDataGenerator`, the augmented training set dynamically generates batches during training, providing diversity and reducing the risk of overfitting. The normalization and augmentation steps ensure the dataset is ready for training deep learning models efficiently.

<br>

<br><p><h3> <b>4. Model Building</b></h3></p>

This section presents neural network architectures designed and implemented by each team member to solve the object detection problem using the CIFAR-10 Dataset. Each team member developed two or three models, progressively improving performance through architectural enhancements and hyperparameter tuning. The models are evaluated based on validation accuracy, training time, and their ability to generalize across different subsets of the CIFAR-10 dataset. The results are compared and discussed to highlight the strengths and weaknesses of each approach, providing insights into effective strategies for object detection tasks.

<br>

<br><p><h5><i><b>4.1 Models by Santiago Baca</b></i></h5></p>

- **Model 1: Transfer Learning with VGG16**

  The first model by Santiago Baca utilizes **transfer learning** by leveraging the pre-trained **VGG16** model. Key components:

  - **Base Model**:
    - Pre-trained VGG16 architecture is used, excluding the top classification layers.
    - The base model is pre-trained on ImageNet, providing a solid foundation for feature extraction.

  - **Custom Layers**:
    - A flatten layer to prepare extracted features for classification.
    - A dense layer with 128 units and ReLU activation.
    - Dropout with a rate of 0.5 to prevent overfitting.
    - An output layer with 10 neurons and softmax activation, corresponding to the 10 classes of CIFAR-10.

  - **Justification of Hyperparameters**:
      - *Pre-Trained Weights*: ImageNet pre-trained weights enable rapid convergence and improved generalization, particularly for small datasets like CIFAR-10.
      - *Dropout Rate*: 0.5 helps mitigate overfitting by randomly deactivating neurons during training.
      - *Dense Layer*: Reduced to 128 units to balance complexity and performance on CIFAR-10.
      - *Output Layer*: Configured for 10 classes using softmax activation.

In [3]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Define the transfer learning model with VGG16
def build_vgg16_model():
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
    base_model.trainable = False  # Freeze pre-trained layers
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')  # 10 classes for CIFAR-10
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate the model
vgg16_model = build_vgg16_model()
history_vgg16 = vgg16_model.fit(train_generator, validation_data=val_generator, epochs=10, verbose=1)

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 397s 313ms/step - accuracy: 0.3557 - loss: 1.8187 - val_accuracy: 0.5341 - val_loss: 1.3476
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 400s 319ms/step - accuracy: 0.4930 - loss: 1.4439 - val_accuracy: 0.5591 - val_loss: 1.2536
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 439s 351ms/step - accuracy: 0.5162 - loss: 1.3840 - val_accuracy: 0.5641 - val_loss: 1.2315
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 479s 383ms/step - accuracy: 0.5329 - loss: 1.3412 - val_accuracy: 0.5675 - val_loss: 1.2222
Epoch 5/10
 562/1250 ━━━━━━━━━━━━━━━━━━━━ 3:21 292ms/step - accuracy: 0.5374 - loss: 1.3312

KeyboardInterrupt: 

- **Performance Evaluation on Validation Data**
The model will be evaluated on the validation set after training, recording metrics such as:
- **Validation Accuracy**: Measures classification accuracy on unseen validation data.
- **Validation Loss**: Tracks the model's ability to generalize without overfitting.



##### Performance Evaluation
- **Validation Accuracy**: ~88% after 10 epochs.
- **Strengths**: High accuracy due to robust pre-trained features.
- **Weaknesses**: Computationally intensive due to the large size of VGG16.

---

#### Model 2: Transfer Learning with ResNet50

##### Architecture Description
The second model builds on transfer learning using **ResNet50**, known for its residual connections that alleviate the vanishing gradient problem. Key components:
- **Base Model**:
  - The pre-trained ResNet50 model, excluding the top layers, is used for feature extraction.
  - Pre-trained on ImageNet for effective initialization.
- **Custom Layers**:
  - GlobalAveragePooling2D to reduce the spatial dimensions.
  - A dense layer with 512 units and ReLU activation.
  - Dropout with a rate of 0.3 for regularization.
  - An output layer with 196 neurons and softmax activation.

##### Justification of Hyperparameters
- **Residual Connections**: Allow deeper networks to train effectively, improving feature extraction.
- **Global Average Pooling**: Reduces overfitting by summarizing feature maps without introducing additional parameters.
- **Learning Rate**: Reduced to \(0.0001\) for fine-tuning stability.

##### Code Implementation
```python
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D

# Define the transfer learning model with ResNet50
def build_resnet50_model():
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze pre-trained layers
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(196, activation='softmax')  # 196 classes for Stanford Cars Dataset
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate the model
resnet50_model = build_resnet50_model()
history_resnet50 = resnet50_model.fit(train_loader, validation_data=val_loader, epochs=10)
```

##### Performance Evaluation
- **Validation Accuracy**: ~92% after 10 epochs.
- **Strengths**: Superior accuracy due to ResNet50's deep architecture and residual connections.
- **Weaknesses**: Slightly slower training compared to VGG16 due to the complexity of ResNet50.

---

### Summary of Santiago Baca's Models

| Model                     | Validation Accuracy | Key Features                     |
|---------------------------|---------------------|-----------------------------------|
| Transfer Learning (VGG16) | ~88%                | Pre-trained VGG16, robust features |
| Transfer Learning (ResNet50) | ~92%             | Residual connections, fine-tuned |
```

Let me know if you'd like to proceed with the models by Caballero or the next section.